<a href="https://colab.research.google.com/github/Diego-II/MDP-Project/blob/main/Notebook/Proyecto_final_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geography based personality characterization

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyspark
!pip install datapackage
!conda install -c plotly plotly-orca

In [79]:
# Librerías 

# Para manipulación de datos
import pandas as pd
import numpy as np
import sys
from pyspark.sql import SparkSession
import time
import plotly.express as px
# import geopandas
import plotly.graph_objects as go
import plotly
plotly.io.orca.config.executable = ''

In [21]:
path = "/content/drive/MyDrive/Procesamiento masivo de datos/Datos/"

## Inquisitive - Curious

In [23]:
spark = SparkSession.builder.appName("BigFiveAnalysis").getOrCreate()
df = spark.read.csv(path + 'data-final.csv', sep ='\t', header = True)
df_5 = df.select('OPN1', 'OPN3', 'OPN5', 'OPN7', 'OPN8', 'OPN9', 'OPN10', 'OPN2', 'OPN4', 'OPN6', 'country').groupBy('country').agg({'OPN1': 'mean', 'OPN3': 'mean', 'OPN5': 'mean', 'OPN7': 'mean',
                                                                                                                                     'OPN8': 'mean', 'OPN9': 'mean','OPN9': 'mean', 'OPN10': 'mean',
                                                                                                                                    'OPN2': 'mean', 'OPN4': 'mean', 'OPN6': 'mean'})
try:
  df_5.coalesce(1).write.csv(path + 'inquisitive_curious', header = True)
except:
  pass

In [24]:
del df, df_5

In [6]:
inquisitive_curious = pd.read_csv(path + 'inquisitive_curious/' + 'part-00000-9e257523-3f65-4634-9acd-83bb1a319e93-c000.csv')

In [25]:
import datapackage

data_url = 'https://datahub.io/core/country-list/datapackage.json'

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources
for resource in resources:
    if resource.tabular:
        country_cod = pd.read_csv(resource.descriptor['path'])
        country_cod.columns = ['Country_name', 'country']

In [26]:
data_url = 'https://datahub.io/core/country-codes/datapackage.json'

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources
for resource in resources:
    if resource.tabular:
        geo_data = pd.read_csv(resource.descriptor['path'])
        geo_data = geo_data.loc[:,['ISO3166-1-Alpha-3', 'ISO3166-1-Alpha-2']]
        geo_data.columns = ['code_3', 'country']

In [27]:
df_incu = pd.merge(left=inquisitive_curious, right=country_cod, how='inner', on='country')
df_incu = pd.merge(left= df_incu, right= geo_data, how='inner', on='country')

In [28]:
df_incu.head()

,country,avg(OPN10),avg(OPN8),avg(OPN2),avg(OPN7),avg(OPN3),avg(OPN4),avg(OPN6),avg(OPN5),avg(OPN9),avg(OPN1),Country_name,code_3
0,LT,3.743869,3.269755,2.073569,3.903724,4.089010,2.073569,1.855586,3.506812,4.092643,3.704814,Lithuania,LTU
1,MM,3.755814,2.558140,2.325581,4.127907,3.627907,2.209302,2.290698,3.651163,3.779070,3.395349,Myanmar,MMR
2,DZ,4.004184,3.133891,1.799163,4.029289,4.121339,1.958159,1.656904,3.887029,4.213389,3.606695,Algeria,DZA
3,CI,3.769231,3.538462,1.615385,4.153846,3.846154,1.846154,2.307692,4.153846,4.076923,4.153846,Côte d'Ivoire,CIV
4,TC,3.142857,2.285714,2.428571,3.857143,4.428571,2.285714,2.428571,3.000000,4.142857,3.857143,Turks and Caicos Islands,TCA


## Amap

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations = df_incu['code_3'],
    z = df_incu['avg(OPN10)'],
    text = df_incu['Country_name'],
    name= True,
    colorscale = 'Greens',
    autocolorscale = False,
    reversescale = False,
    marker_line_color = 'darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Big Five-factor v',
))

fig.update_layout(
    title_text='Inquisitive - Curious',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)

fig.show()

## PIG MAPS

In [29]:
path2 = "/content/drive/MyDrive/Procesamiento masivo de datos/Datos_PIG/"
agg_big5 = pd.read_csv(path2 + 'agg_big5.tsv', sep='\t', skiprows=(1, 2, 3, 4))
agg_country_big5 = pd.read_csv(path2 + 'agg_country_big5.tsv', sep='\t', skiprows=(1,2))
agg_country_big5 = agg_country_big5.iloc[0:224]

In [30]:
agg_big5_new = pd.merge(left = agg_big5, right = country_cod, how ='inner', on ='country')
agg_big5_new = pd.merge(left = agg_big5_new, right= geo_data, how ='inner', on = 'country')

In [31]:
agg_big5_new['latitud'] = agg_big5_new.latitud.astype(float)
agg_big5_new['longitud'] = agg_big5_new.longitud.astype(float)

In [32]:
agg_big5_new.head()

,EXT,EST,AGR,CSN,OPN,country,latitud,longitud,Country_name,code_3
0,16.0,12.0,15.0,8.0,27.0,GB,51.5448,0.1991,United Kingdom,GBR
1,-5.0,14.0,18.0,10.0,23.0,GB,54.9119,-1.3833,United Kingdom,GBR
2,-4.0,17.0,14.0,1.0,21.0,GB,51.7500,-1.2500,United Kingdom,GBR
3,2.0,18.0,21.0,7.0,27.0,GB,53.4230,-2.2166,United Kingdom,GBR
4,-20.0,32.0,2.0,15.0,24.0,GB,53.8000,-1.5833,United Kingdom,GBR


In [ ]:
fig = px.scatter_mapbox(agg_big5_new, lat = "latitud", lon = "longitud", color = 'EXT', color_continuous_scale = 'icefire', height=800, zoom=3)

fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom = 1.5, margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [34]:
agg_country_big5_new = pd.merge(left = agg_country_big5, right = country_cod, how ='inner', on ='country')
agg_country_big5_new = pd.merge(left = agg_country_big5_new, right= geo_data, how ='inner', on = 'country')

In [36]:
agg_country_big5_new.head() 

,EXT,EST,AGR,CSN,OPN,count_res,country,Country_name,code_3
0,-0.294118,16.411765,14.294118,9.470588,20.705882,17,AD,Andorra,AND
1,0.312745,19.256863,13.633987,9.927451,19.201307,3060,AE,United Arab Emirates,ARE
2,1.018519,16.259259,12.222222,10.240741,18.500000,54,AF,Afghanistan,AFG
3,-2.666667,18.500000,11.541667,9.625000,21.916667,24,AG,Antigua and Barbuda,ATG
4,-2.250000,18.250000,18.500000,12.250000,24.500000,4,AI,Anguilla,AIA


In [46]:
!curl 'https://www.idea.int/gsod-indices/sites/default/files/inline-files/GSoDI%20v5%20%281975-2020%29.csv' >> extra_data.csv
!ls

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 23.2M  100 23.2M    0     0  14.6M      0  0:00:01  0:00:01 --:--:-- 14.6M
drive  extra_data.csv  GSoDI%20v5%20%281975-2020%29.csv  sample_data


In [50]:
cols_a_usar = ["ID_country_name", "ID_year","regime_status_name", "democratic_performance_name", "democratic_performance_numeric"]
extra = pd.read_csv('extra_data.csv', usecols=cols_a_usar)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (2,200,201,202) have mixed types.Specify dtype option on import or set low_memory=False.



In [58]:
df_extra = extra[extra["ID_year"] == 2020]
df_extra.rename(columns = {'ID_country_name':'Country_name'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [60]:
df_extra.head()

,Country_name,ID_year,regime_status_name,democratic_performance_name,democratic_performance_numeric
45,United States,2020,Democracy,High performing democracy,1
91,Canada,2020,Democracy,Mid-range performing democracy,2
137,Cuba,2020,Authoritarian Regime,Authoritarian Regime,5
183,Haiti,2020,Hybrid Regime,Hybrid Regime,4
229,Dominican Republic,2020,Democracy,Mid-range performing democracy,2


In [62]:
big5_extra_info = pd.merge(left = agg_country_big5_new, right = df_extra, how ='inner', on ='Country_name')

In [64]:
big5_extra_info.head()

,EXT,EST,AGR,CSN,OPN,count_res,country,Country_name,code_3,ID_year,regime_status_name,democratic_performance_name,democratic_performance_numeric
0,0.312745,19.256863,13.633987,9.927451,19.201307,3060,AE,United Arab Emirates,ARE,2020,Authoritarian Regime,Authoritarian Regime,5
1,0.312745,19.256863,13.633987,9.927451,19.201307,3060,AE,United Arab Emirates,ARE,2020,Authoritarian Regime,Authoritarian Regime,5
2,0.312745,19.256863,13.633987,9.927451,19.201307,3060,AE,United Arab Emirates,ARE,2020,Authoritarian Regime,Authoritarian Regime,5
3,1.018519,16.259259,12.222222,10.240741,18.500000,54,AF,Afghanistan,AFG,2020,Hybrid Regime,Hybrid Regime,4
4,1.018519,16.259259,12.222222,10.240741,18.500000,54,AF,Afghanistan,AFG,2020,Hybrid Regime,Hybrid Regime,4


In [66]:
big5_extra_info.columns

Index(['EXT', 'EST', 'AGR', 'CSN', 'OPN', 'count_res', 'country',
       'Country_name', 'code_3', 'ID_year', 'regime_status_name',
       'democratic_performance_name', 'democratic_performance_numeric'],
      dtype='object')

In [96]:
for i in range(1,6):
  print(f"""
  democratic_performance_numeric: {i}, regime_status_name: {big5_extra_info[big5_extra_info["democratic_performance_numeric"] == i]["regime_status_name"].reset_index().loc[0]},
  democratic_performance_name: {big5_extra_info[big5_extra_info["democratic_performance_numeric"] == i]["democratic_performance_name"].reset_index().loc[0]}
  """)


  democratic_performance_numeric: 1, regime_status_name: index                        12
regime_status_name    Democracy
Name: 0, dtype: object,
  democratic_performance_name: index                                                 12
democratic_performance_name    High performing democracy
Name: 0, dtype: object
  

  democratic_performance_numeric: 2, regime_status_name: index                        11
regime_status_name    Democracy
Name: 0, dtype: object,
  democratic_performance_name: index                                                      11
democratic_performance_name    Mid-range performing democracy
Name: 0, dtype: object
  

  democratic_performance_numeric: 3, regime_status_name: index                         6
regime_status_name    Democracy
Name: 0, dtype: object,
  democratic_performance_name: index                                       6
democratic_performance_name    Weak democracy
Name: 0, dtype: object
  

  democratic_performance_numeric: 4, regime_status_name: ind

In [91]:
fig = go.Figure(data=go.Choropleth(
    locations = big5_extra_info['code_3'],
    z = big5_extra_info['democratic_performance_numeric'],
    text = big5_extra_info['Country_name'],
    colorscale = 'Viridis',
    autocolorscale = False,
    reversescale = False,
    marker_line_color = 'darkgray',
    marker_line_width=0.5,
    colorbar_title = "Democratic Performance <br> Less is better.",
))

fig.update_layout(
    title_text='Democratic Performance',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)



fig.show()


In [65]:
fig = go.Figure(data=go.Choropleth(
    locations = agg_country_big5_new['code_3'],
    z = agg_country_big5_new['EXT'],
    text = agg_country_big5_new['Country_name'],
    colorscale = 'icefire',
    autocolorscale = False,
    reversescale = False,
    marker_line_color = 'darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Extraversion level',
))

fig.update_layout(
    title_text='Extraversion per Country',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)

fig.show()


In [97]:
agg_country_big5_new[agg_country_big5_new['count_res']>200].sort_values(by = 'EXT')

,EXT,EST,AGR,CSN,OPN,count_res,country,Country_name,code_3
56,-3.748954,21.234310,10.702929,7.615063,21.581590,239,DZ,Algeria,DZA
123,-3.727455,19.805611,10.993988,7.727455,21.851703,499,MA,Morocco,MAR
29,-3.346579,19.452011,10.853549,7.148694,22.298189,6241,BR,Brazil,BRA
18,-3.326019,19.752351,12.134796,7.940439,18.109718,319,BD,Bangladesh,BGD
74,-3.154412,16.488971,14.643382,12.922794,19.363971,272,GH,Ghana,GHA
...,...,...,...,...,...,...,...,...,...
53,0.936557,18.355753,13.559353,8.424105,21.424286,5501,DK,Denmark,DNK
157,0.964912,17.035088,13.785088,11.206140,19.942982,228,OM,Oman,OMN
152,0.967270,17.139102,13.728547,8.978112,21.421704,9777,NL,Netherlands,NLD
153,0.989608,17.696445,13.786418,9.607384,21.166727,10970,NO,Norway,NOR


In [98]:
fig = go.Figure(data=go.Choropleth(
    locations = agg_country_big5_new['code_3'],
    z = agg_country_big5_new['AGR'],
    text = agg_country_big5_new['Country_name'],
    colorscale = 'icefire',
    autocolorscale = False,
    reversescale = False,
    marker_line_color = 'darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Agreeableness level',
))

fig.update_layout(
    title_text='Agreeableness per Country',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)

fig.show()


In [99]:
agg_country_big5_new[agg_country_big5_new['count_res']>200].sort_values(by = 'AGR')

,EXT,EST,AGR,CSN,OPN,count_res,country,Country_name,code_3
164,-2.714040,20.543152,9.818162,6.964148,22.676041,4658,PL,Poland,POL
119,-2.473206,19.726612,10.266122,7.600363,20.307902,1101,LT,Lithuania,LTU
121,-1.624758,19.195358,10.363636,7.646035,21.048356,517,LV,Latvia,LVA
175,-2.000903,20.451972,10.496838,7.991268,20.669377,3321,RU,Russian Federation,RUS
71,-1.055710,19.935933,10.623955,8.275766,21.504178,359,GE,Georgia,GEO
...,...,...,...,...,...,...,...,...,...
209,-0.155063,18.306330,14.191549,10.047759,20.851641,545912,US,United States,USA
74,-3.154412,16.488971,14.643382,12.922794,19.363971,272,GH,Ghana,GHA
150,-1.292017,17.290966,14.967437,12.022059,20.567227,952,NG,Nigeria,NGA
103,-1.710320,17.467616,15.044128,12.111744,20.861922,1405,KE,Kenya,KEN


In [100]:
fig = go.Figure(data=go.Choropleth(
    locations = agg_country_big5_new['code_3'],
    z = agg_country_big5_new['CSN'],
    text = agg_country_big5_new['Country_name'],
    colorscale = 'icefire',
    autocolorscale = False,
    reversescale = False,
    marker_line_color = 'darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Conscientiousness level',
))

fig.update_layout(
    title_text='Conscientiousness per Country',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)

fig.show()

In [101]:
agg_country_big5_new[agg_country_big5_new['count_res']>200].sort_values(by = 'CSN')

,EXT,EST,AGR,CSN,OPN,count_res,country,Country_name,code_3
27,-1.027397,20.701370,10.926027,6.057534,20.701370,365,BO,"Bolivia, Plurinational State of",BOL
170,-1.412811,20.512456,11.615658,6.697509,21.391459,281,PY,Paraguay,PRY
164,-2.714040,20.543152,9.818162,6.964148,22.676041,4658,PL,Poland,POL
159,-0.663024,20.006679,11.420765,7.043109,20.241651,1647,PE,Peru,PER
29,-3.346579,19.452011,10.853549,7.148694,22.298189,6241,BR,Brazil,BRA
...,...,...,...,...,...,...,...,...,...
157,0.964912,17.035088,13.785088,11.206140,19.942982,228,OM,Oman,OMN
61,1.342960,14.548736,15.231047,11.880866,19.292419,277,ET,Ethiopia,ETH
150,-1.292017,17.290966,14.967437,12.022059,20.567227,952,NG,Nigeria,NGA
103,-1.710320,17.467616,15.044128,12.111744,20.861922,1405,KE,Kenya,KEN


In [102]:
fig = go.Figure(data=go.Choropleth(
    locations = agg_country_big5_new['code_3'],
    z = agg_country_big5_new['EST'],
    text = agg_country_big5_new['Country_name'],
    colorscale = 'icefire',
    autocolorscale = False,
    reversescale = False,
    marker_line_color = 'darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Neuroticism level',
))

fig.update_layout(
    title_text='Neuroticism per Country',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)

fig.show()

In [103]:
agg_country_big5_new[agg_country_big5_new['count_res']>200].sort_values(by = 'EST')

,EXT,EST,AGR,CSN,OPN,count_res,country,Country_name,code_3
61,1.342960,14.548736,15.231047,11.880866,19.292419,277,ET,Ethiopia,ETH
74,-3.154412,16.488971,14.643382,12.922794,19.363971,272,GH,Ghana,GHA
157,0.964912,17.035088,13.785088,11.206140,19.942982,228,OM,Oman,OMN
152,0.967270,17.139102,13.728547,8.978112,21.421704,9777,NL,Netherlands,NLD
150,-1.292017,17.290966,14.967437,12.022059,20.567227,952,NG,Nigeria,NGA
...,...,...,...,...,...,...,...,...,...
27,-1.027397,20.701370,10.926027,6.057534,20.701370,365,BO,"Bolivia, Plurinational State of",BOL
81,-0.990048,20.870621,12.494427,8.583599,21.618232,2512,GR,Greece,GRC
59,-1.340755,20.886738,12.785092,8.762827,19.428848,1033,EG,Egypt,EGY
22,-1.173913,21.086957,12.647343,9.159420,19.695652,207,BH,Bahrain,BHR


In [104]:
fig = go.Figure(data=go.Choropleth(
    locations = agg_country_big5_new['code_3'],
    z = agg_country_big5_new['OPN'],
    text = agg_country_big5_new['Country_name'],
    colorscale = 'icefire',
    autocolorscale = False,
    reversescale = False,
    marker_line_color = 'darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Openness level',
))

fig.update_layout(
    title_text='Openness per Country',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)

fig.show()

In [105]:
agg_country_big5_new[agg_country_big5_new['count_res']>200].sort_values(by = 'OPN')

,EXT,EST,AGR,CSN,OPN,count_res,country,Country_name,code_3
134,-0.708609,18.966887,12.642384,8.933775,15.506623,302,MO,Macao,MAC
105,-1.516883,20.327273,11.324675,8.851948,15.971429,385,KH,Cambodia,KHM
144,-1.341412,19.544373,12.074645,8.970301,16.221204,11347,MY,Malaysia,MYS
154,-0.828660,19.300623,12.031153,7.841121,16.783489,642,NP,Nepal,NPL
26,-2.878543,20.489879,11.821862,8.222672,17.546559,247,BN,Brunei Darussalam,BRN
...,...,...,...,...,...,...,...,...,...
174,-0.715738,19.086199,12.034383,8.830508,22.583051,2065,RS,Serbia,SRB
67,-1.142182,19.263192,12.880293,7.934853,22.660586,6140,FR,France,FRA
164,-2.714040,20.543152,9.818162,6.964148,22.676041,4658,PL,Poland,POL
92,0.053110,18.104123,12.948288,9.183788,22.888190,1431,IL,Israel,ISR
